In [1]:
import numpy as np
import os
from tqdm import tqdm
np.seterr(all='raise',invalid='raise')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [2]:
data_path = '/labs/hulab/stark_stuff/ppg_ecg_project/data/'
split = 'train'

In [8]:
# AF_ECG = np.load(f'{data_path}/AF_v5/{split}_ECG_resampled2400.npy')
# PVC_ECG = np.load(f'{data_path}/PVC_v5/{split}_ECG_resampled2400.npy')
# NSR_ECG = np.load(f'{data_path}/NSR_v5/{split}_ECG_resampled2400.npy')

In [19]:
import torch
import torch.nn as nn

# class debug_model(nn.Module):
#     def __init__(self, num_classes=2):
#         super(debug_model,self).__init__()
#         self.num_classes = num_classes
#         self.conv0 = nn.Conv1d(1, 48, 80, 4)
#         self.avgpool = nn.AvgPool1d(1)
#         self.dense1 = nn.Linear(384, self.num_classes)
        
#     def forward(self, x):
#         out = self.conv0(x)
#         out = self.avgpool(out)
#         out = self.dense1(features)

class ResBlock(nn.Module):

    def __init__(self, filters, stage, i_block, kernel_size=3, stride=1):
        super(ResBlock,self).__init__()
        self.out_channels = filters
        self.kernel_size = kernel_size
        self.stride = stride
        self.in_channels = filters

        if i_block == 0 and stage != 0:
            self.in_channels = filters // 2

        self.conv1 = nn.Conv1d(
            in_channels=self.in_channels,
            out_channels=self.out_channels,
            kernel_size=self.kernel_size,
            stride=self.stride,
            padding=1)
        self.bn1 = nn.BatchNorm1d(self.out_channels)
        self.relu1 = nn.ReLU()

        self.conv2 = nn.Conv1d(
            in_channels=self.out_channels,
            out_channels=self.out_channels,
            kernel_size=self.kernel_size,
            stride=self.stride,
            padding=1)

        self.bn2 = nn.BatchNorm1d(self.out_channels)
        self.bn3 = nn.BatchNorm1d(self.out_channels)
        self.relu2 = nn.ReLU()

    def forward(self, x):

        out = x
        out = self.relu1(self.bn1(self.conv1(out)))
        out = self.bn2(self.conv2(out))
        if x.shape[1] != out.shape[1]:
            out = torch.add(out, torch.repeat_interleave(x, 2, dim=1))
        else:
            out = torch.add(out, x)

        out = self.relu2(self.bn3(out))

        return out


class Resnet34(nn.Module):
    def __init__(self, num_classes=2):
        super(Resnet34,self).__init__()
        self.num_classes = num_classes
        self.conv0 = nn.Conv1d(1, 48, 80, 4)
        self.bn0 = nn.BatchNorm1d(48)
        self.relu = nn.ReLU()
        self.pool0 = nn.MaxPool1d(4)
        self.stage0 = nn.Sequential(ResBlock(48, 0, 0),
                                   ResBlock(48, 0, 1))
        self.pool1 = nn.MaxPool1d(4)
        self.stage1 = nn.Sequential(ResBlock(96, 1, 0),
                                   ResBlock(96, 1, 1))
        self.pool2 = nn.MaxPool1d(4)
        self.stage2 = nn.Sequential(ResBlock(192, 2, 0),
                                    ResBlock(192, 2, 1))
        self.pool3 = nn.MaxPool1d(4)
        self.stage3 = nn.Sequential(ResBlock(384, 3, 0),
                                    ResBlock(384, 3, 1))

        self.avgpool = nn.AvgPool1d(1)
        self.dense1 = nn.Linear(384, self.num_classes)

    def forward(self, x):
        
        out = self.conv0(x)
        out = self.bn0(out)
        out = self.relu(out)
        out = self.pool0(out)


        out = self.stage0(out)
        out = self.pool1(out)

        out = self.stage1(out)
        out = self.pool2(out)


        out = self.stage2(out)
        out = self.pool3(out)


        out = self.stage3(out)


        out = self.avgpool(out)
        features = out.mean(dim=2)

        out = self.dense1(features)

        return features, out




In [21]:
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
import pickle as pkl
import torch
import random
import gc
from datetime import datetime
import psutil

class Dataset_ECG(Dataset):
    def __init__(self, data_path, split, subset):
        super().__init__()
        self.data_path = data_path
        self.split = split
        self.build_dataset()
        self.length = len(self.all_labels)
        self.subset = subset

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        
        if idx < self.AF_ECG.shape[0]:
            return self.AF_ECG[idx][None, :], self.all_labels[idx]
        elif idx >= self.AF_ECG.shape[0] and idx < (self.AF_ECG.shape[0] + self.PVC_ECG.shape[0]):
            offset = self.AF_ECG.shape[0]
            return self.PVC_ECG[idx-offset][None, :], self.all_labels[idx]
        else:
            offset = self.AF_ECG.shape[0] + self.PVC_ECG.shape[0]
            return self.NSR_ECG[idx-offset][None, :], self.all_labels[idx]

    def build_dataset(self):
        tstart = datetime.now()
        print(f'\tloading... {self.split}')
        
        self.AF_ECG = torch.from_numpy(np.load(f'{self.data_path}/AF_v5/{self.split}_ECG_resampled2400.npy')).float()
        print(f'\tAF loaded {self.split}')
        self.PVC_ECG = torch.from_numpy(np.load(f'{self.data_path}/PVC_v5/{self.split}_ECG_resampled2400.npy')).float()
        print(f'\tPVC loaded {self.split}')
        self.NSR_ECG = torch.from_numpy(np.load(f'{self.data_path}/NSR_v5/{self.split}_ECG_resampled2400.npy')).float()
        print(f'\tNSR loaded {self.split}')

        print(f'\tloading {self.split} finished t={datetime.now() - tstart}, mem used={psutil.virtual_memory()[3]/1000000000}')
        
        AF_labels = np.ones((self.AF_ECG.shape[0]))
        NSR_labels = np.zeros((self.PVC_ECG.shape[0] + self.NSR_ECG.shape[0]))
        self.all_labels = torch.from_numpy(np.concatenate((AF_labels, NSR_labels), axis=0)).long()

        print(f'dataset built AF counts {len(AF_labels)}, NSR counts {len(NSR_labels)}, total counts {len(self.all_labels)}', flush=True)

In [11]:
train_dataset = Dataset_ECG('/labs/hulab/stark_stuff/ppg_ecg_project/data/', split='train', subset=0)

	loading... train
	AF loaded train
	PVC loaded train
	NSR loaded train
	loading train finished t=0:01:12.149117, mem used=66.087018496
dataset built AF counts 2757889, NSR counts 3014334, total counts 5772223


In [37]:
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=False, num_workers=4)

In [38]:
import torch.optim as optim

ECG_model = Resnet34().to('cuda')
device = 'cuda'
ECG_LR = 1e-4
ce_loss_fn = nn.CrossEntropyLoss()
ECG_optimizer = optim.Adam(ECG_model.parameters(), lr=ECG_LR)

for batch_idx, data in enumerate(train_loader):
    
    if batch_idx < 81*2500: continue
    
    if batch_idx > 82*2500: break

    ECG, target = data
    ECG = ECG.to(device)
    target = target.to(device)
    ECG_feature, ECG_out = ECG_model(ECG)
    ECG_loss = ce_loss_fn(ECG_out, target)
    
    ECG_optimizer.zero_grad()
    ECG_loss.backward()
    ECG_optimizer.step()
    
    print(batch_idx, ECG_loss)

202500 tensor(0.3943, device='cuda:0', grad_fn=<NllLossBackward>)
202501 tensor(0.3814, device='cuda:0', grad_fn=<NllLossBackward>)
202502 tensor(0.3690, device='cuda:0', grad_fn=<NllLossBackward>)
202503 tensor(0.3569, device='cuda:0', grad_fn=<NllLossBackward>)
202504 tensor(0.3451, device='cuda:0', grad_fn=<NllLossBackward>)
202505 tensor(0.3337, device='cuda:0', grad_fn=<NllLossBackward>)
202506 tensor(0.3227, device='cuda:0', grad_fn=<NllLossBackward>)
202507 tensor(0.3120, device='cuda:0', grad_fn=<NllLossBackward>)
202508 tensor(0.3023, device='cuda:0', grad_fn=<NllLossBackward>)
202509 tensor(0.2916, device='cuda:0', grad_fn=<NllLossBackward>)
202510 tensor(0.2819, device='cuda:0', grad_fn=<NllLossBackward>)
202511 tensor(0.2720, device='cuda:0', grad_fn=<NllLossBackward>)
202512 tensor(0.2636, device='cuda:0', grad_fn=<NllLossBackward>)
202513 tensor(0.2549, device='cuda:0', grad_fn=<NllLossBackward>)
202514 tensor(0.2465, device='cuda:0', grad_fn=<NllLossBackward>)
202515 ten

202629 tensor(0.0282, device='cuda:0', grad_fn=<NllLossBackward>)
202630 tensor(0.0279, device='cuda:0', grad_fn=<NllLossBackward>)
202631 tensor(0.0276, device='cuda:0', grad_fn=<NllLossBackward>)
202632 tensor(0.0273, device='cuda:0', grad_fn=<NllLossBackward>)
202633 tensor(0.0271, device='cuda:0', grad_fn=<NllLossBackward>)
202634 tensor(0.0268, device='cuda:0', grad_fn=<NllLossBackward>)
202635 tensor(0.0265, device='cuda:0', grad_fn=<NllLossBackward>)
202636 tensor(0.0263, device='cuda:0', grad_fn=<NllLossBackward>)
202637 tensor(0.0260, device='cuda:0', grad_fn=<NllLossBackward>)
202638 tensor(0.0258, device='cuda:0', grad_fn=<NllLossBackward>)
202639 tensor(0.0255, device='cuda:0', grad_fn=<NllLossBackward>)
202640 tensor(0.0253, device='cuda:0', grad_fn=<NllLossBackward>)
202641 tensor(0.0251, device='cuda:0', grad_fn=<NllLossBackward>)
202642 tensor(0.0248, device='cuda:0', grad_fn=<NllLossBackward>)
202643 tensor(0.0246, device='cuda:0', grad_fn=<NllLossBackward>)
202644 ten

202760 tensor(0.0108, device='cuda:0', grad_fn=<NllLossBackward>)
202761 tensor(0.0107, device='cuda:0', grad_fn=<NllLossBackward>)
202762 tensor(0.0111, device='cuda:0', grad_fn=<NllLossBackward>)
202763 tensor(0.0106, device='cuda:0', grad_fn=<NllLossBackward>)
202764 tensor(0.0105, device='cuda:0', grad_fn=<NllLossBackward>)
202765 tensor(0.0105, device='cuda:0', grad_fn=<NllLossBackward>)
202766 tensor(0.0104, device='cuda:0', grad_fn=<NllLossBackward>)
202767 tensor(0.0104, device='cuda:0', grad_fn=<NllLossBackward>)
202768 tensor(0.0103, device='cuda:0', grad_fn=<NllLossBackward>)
202769 tensor(0.0103, device='cuda:0', grad_fn=<NllLossBackward>)
202770 tensor(0.0102, device='cuda:0', grad_fn=<NllLossBackward>)
202771 tensor(0.0101, device='cuda:0', grad_fn=<NllLossBackward>)
202772 tensor(0.0101, device='cuda:0', grad_fn=<NllLossBackward>)
202773 tensor(0.0100, device='cuda:0', grad_fn=<NllLossBackward>)
202774 tensor(0.0100, device='cuda:0', grad_fn=<NllLossBackward>)
202775 ten

202891 tensor(0.0059, device='cuda:0', grad_fn=<NllLossBackward>)
202892 tensor(0.0059, device='cuda:0', grad_fn=<NllLossBackward>)
202893 tensor(0.0059, device='cuda:0', grad_fn=<NllLossBackward>)
202894 tensor(0.0059, device='cuda:0', grad_fn=<NllLossBackward>)
202895 tensor(0.0058, device='cuda:0', grad_fn=<NllLossBackward>)
202896 tensor(0.0058, device='cuda:0', grad_fn=<NllLossBackward>)
202897 tensor(0.0058, device='cuda:0', grad_fn=<NllLossBackward>)
202898 tensor(0.0058, device='cuda:0', grad_fn=<NllLossBackward>)
202899 tensor(0.0057, device='cuda:0', grad_fn=<NllLossBackward>)
202900 tensor(0.0057, device='cuda:0', grad_fn=<NllLossBackward>)
202901 tensor(0.0057, device='cuda:0', grad_fn=<NllLossBackward>)
202902 tensor(0.0057, device='cuda:0', grad_fn=<NllLossBackward>)
202903 tensor(0.0057, device='cuda:0', grad_fn=<NllLossBackward>)
202904 tensor(0.0056, device='cuda:0', grad_fn=<NllLossBackward>)
202905 tensor(0.0056, device='cuda:0', grad_fn=<NllLossBackward>)
202906 ten

203016 tensor(0.0039, device='cuda:0', grad_fn=<NllLossBackward>)
203017 tensor(0.0039, device='cuda:0', grad_fn=<NllLossBackward>)
203018 tensor(0.0039, device='cuda:0', grad_fn=<NllLossBackward>)
203019 tensor(0.0039, device='cuda:0', grad_fn=<NllLossBackward>)
203020 tensor(0.0038, device='cuda:0', grad_fn=<NllLossBackward>)
203021 tensor(0.0038, device='cuda:0', grad_fn=<NllLossBackward>)
203022 tensor(0.0038, device='cuda:0', grad_fn=<NllLossBackward>)
203023 tensor(0.0038, device='cuda:0', grad_fn=<NllLossBackward>)
203024 tensor(0.0038, device='cuda:0', grad_fn=<NllLossBackward>)
203025 tensor(0.0038, device='cuda:0', grad_fn=<NllLossBackward>)
203026 tensor(0.0038, device='cuda:0', grad_fn=<NllLossBackward>)
203027 tensor(0.0038, device='cuda:0', grad_fn=<NllLossBackward>)
203028 tensor(0.0037, device='cuda:0', grad_fn=<NllLossBackward>)
203029 tensor(0.0037, device='cuda:0', grad_fn=<NllLossBackward>)
203030 tensor(0.0037, device='cuda:0', grad_fn=<NllLossBackward>)
203031 ten

...

204458 tensor(nan, device='cuda:0', grad_fn=<NllLossBackward>)
204459 tensor(nan, device='cuda:0', grad_fn=<NllLossBackward>)
204460 tensor(nan, device='cuda:0', grad_fn=<NllLossBackward>)
204461 tensor(nan, device='cuda:0', grad_fn=<NllLossBackward>)
204462 tensor(nan, device='cuda:0', grad_fn=<NllLossBackward>)
204463 tensor(nan, device='cuda:0', grad_fn=<NllLossBackward>)
204464 tensor(nan, device='cuda:0', grad_fn=<NllLossBackward>)
204465 tensor(nan, device='cuda:0', grad_fn=<NllLossBackward>)
204466 tensor(nan, device='cuda:0', grad_fn=<NllLossBackward>)
204467 tensor(nan, device='cuda:0', grad_fn=<NllLossBackward>)
204468 tensor(nan, device='cuda:0', grad_fn=<NllLossBackward>)
204469 tensor(nan, device='cuda:0', grad_fn=<NllLossBackward>)
204470 tensor(nan, device='cuda:0', grad_fn=<NllLossBackward>)
204471 tensor(nan, device='cuda:0', grad_fn=<NllLossBackward>)
204472 tensor(nan, device='cuda:0', grad_fn=<NllLossBackward>)
204473 tensor(nan, device='cuda:0', grad_fn=<NllLossBac

204591 tensor(nan, device='cuda:0', grad_fn=<NllLossBackward>)
204592 tensor(nan, device='cuda:0', grad_fn=<NllLossBackward>)
204593 tensor(nan, device='cuda:0', grad_fn=<NllLossBackward>)
204594 tensor(nan, device='cuda:0', grad_fn=<NllLossBackward>)
204595 tensor(nan, device='cuda:0', grad_fn=<NllLossBackward>)
204596 tensor(nan, device='cuda:0', grad_fn=<NllLossBackward>)
204597 tensor(nan, device='cuda:0', grad_fn=<NllLossBackward>)
204598 tensor(nan, device='cuda:0', grad_fn=<NllLossBackward>)
204599 tensor(nan, device='cuda:0', grad_fn=<NllLossBackward>)
204600 tensor(nan, device='cuda:0', grad_fn=<NllLossBackward>)
204601 tensor(nan, device='cuda:0', grad_fn=<NllLossBackward>)
204602 tensor(nan, device='cuda:0', grad_fn=<NllLossBackward>)
204603 tensor(nan, device='cuda:0', grad_fn=<NllLossBackward>)
204604 tensor(nan, device='cuda:0', grad_fn=<NllLossBackward>)
204605 tensor(nan, device='cuda:0', grad_fn=<NllLossBackward>)
204606 tensor(nan, device='cuda:0', grad_fn=<NllLossBac

204722 tensor(nan, device='cuda:0', grad_fn=<NllLossBackward>)
204723 tensor(nan, device='cuda:0', grad_fn=<NllLossBackward>)
204724 tensor(nan, device='cuda:0', grad_fn=<NllLossBackward>)
204725 tensor(nan, device='cuda:0', grad_fn=<NllLossBackward>)
204726 tensor(nan, device='cuda:0', grad_fn=<NllLossBackward>)
204727 tensor(nan, device='cuda:0', grad_fn=<NllLossBackward>)
204728 tensor(nan, device='cuda:0', grad_fn=<NllLossBackward>)
204729 tensor(nan, device='cuda:0', grad_fn=<NllLossBackward>)
204730 tensor(nan, device='cuda:0', grad_fn=<NllLossBackward>)
204731 tensor(nan, device='cuda:0', grad_fn=<NllLossBackward>)
204732 tensor(nan, device='cuda:0', grad_fn=<NllLossBackward>)
204733 tensor(nan, device='cuda:0', grad_fn=<NllLossBackward>)
204734 tensor(nan, device='cuda:0', grad_fn=<NllLossBackward>)
204735 tensor(nan, device='cuda:0', grad_fn=<NllLossBackward>)
204736 tensor(nan, device='cuda:0', grad_fn=<NllLossBackward>)
204737 tensor(nan, device='cuda:0', grad_fn=<NllLossBac

204857 tensor(nan, device='cuda:0', grad_fn=<NllLossBackward>)
204858 tensor(nan, device='cuda:0', grad_fn=<NllLossBackward>)
204859 tensor(nan, device='cuda:0', grad_fn=<NllLossBackward>)
204860 tensor(nan, device='cuda:0', grad_fn=<NllLossBackward>)
204861 tensor(nan, device='cuda:0', grad_fn=<NllLossBackward>)
204862 tensor(nan, device='cuda:0', grad_fn=<NllLossBackward>)
204863 tensor(nan, device='cuda:0', grad_fn=<NllLossBackward>)
204864 tensor(nan, device='cuda:0', grad_fn=<NllLossBackward>)
204865 tensor(nan, device='cuda:0', grad_fn=<NllLossBackward>)
204866 tensor(nan, device='cuda:0', grad_fn=<NllLossBackward>)
204867 tensor(nan, device='cuda:0', grad_fn=<NllLossBackward>)
204868 tensor(nan, device='cuda:0', grad_fn=<NllLossBackward>)
204869 tensor(nan, device='cuda:0', grad_fn=<NllLossBackward>)
204870 tensor(nan, device='cuda:0', grad_fn=<NllLossBackward>)
204871 tensor(nan, device='cuda:0', grad_fn=<NllLossBackward>)
204872 tensor(nan, device='cuda:0', grad_fn=<NllLossBac

204990 tensor(nan, device='cuda:0', grad_fn=<NllLossBackward>)
204991 tensor(nan, device='cuda:0', grad_fn=<NllLossBackward>)
204992 tensor(nan, device='cuda:0', grad_fn=<NllLossBackward>)
204993 tensor(nan, device='cuda:0', grad_fn=<NllLossBackward>)
204994 tensor(nan, device='cuda:0', grad_fn=<NllLossBackward>)
204995 tensor(nan, device='cuda:0', grad_fn=<NllLossBackward>)
204996 tensor(nan, device='cuda:0', grad_fn=<NllLossBackward>)
204997 tensor(nan, device='cuda:0', grad_fn=<NllLossBackward>)
204998 tensor(nan, device='cuda:0', grad_fn=<NllLossBackward>)
204999 tensor(nan, device='cuda:0', grad_fn=<NllLossBackward>)
205000 tensor(nan, device='cuda:0', grad_fn=<NllLossBackward>)


In [31]:
203888 -> 203891

200000

In [39]:
import torch.optim as optim

ECG_model = Resnet34().to('cuda')
device = 'cuda'
ECG_LR = 1e-4
ce_loss_fn = nn.CrossEntropyLoss()
ECG_optimizer = optim.Adam(ECG_model.parameters(), lr=ECG_LR)

for batch_idx, data in enumerate(train_loader):
    
    if batch_idx < 203888: continue
    
    if batch_idx > 203892: break

    ECG, target = data
    ECG = ECG.to(device)
    target = target.to(device)
    ECG_feature, ECG_out = ECG_model(ECG)
    ECG_loss = ce_loss_fn(ECG_out, target)
    
    ECG_optimizer.zero_grad()
    ECG_loss.backward()
    ECG_optimizer.step()
    
    print(batch_idx, ECG_loss)

203888 tensor(0.8182, device='cuda:0', grad_fn=<NllLossBackward>)
203889 tensor(nan, device='cuda:0', grad_fn=<NllLossBackward>)
203890 tensor(nan, device='cuda:0', grad_fn=<NllLossBackward>)
203891 tensor(nan, device='cuda:0', grad_fn=<NllLossBackward>)
203892 tensor(nan, device='cuda:0', grad_fn=<NllLossBackward>)


In [42]:
import torch.optim as optim

ECG_model = Resnet34().to('cuda')
device = 'cuda'
ECG_LR = 1e-4
ce_loss_fn = nn.CrossEntropyLoss()
ECG_optimizer = optim.Adam(ECG_model.parameters(), lr=ECG_LR)

for batch_idx, data in enumerate(train_loader):
    ECG, target = data
    ECG = ECG.to(device)
    target = target.to(device)
    ECG_feature, ECG_out = ECG_model(ECG)
    ECG_loss = ce_loss_fn(ECG_out, target)
    
    ECG_optimizer.zero_grad()
    ECG_loss.backward()
    ECG_optimizer.step()
    
    print(batch_idx, ECG_loss)

0 tensor(0.8040, device='cuda:0', grad_fn=<NllLossBackward>)
1 tensor(0.7821, device='cuda:0', grad_fn=<NllLossBackward>)
2 tensor(0.7607, device='cuda:0', grad_fn=<NllLossBackward>)
3 tensor(0.7397, device='cuda:0', grad_fn=<NllLossBackward>)
4 tensor(0.7191, device='cuda:0', grad_fn=<NllLossBackward>)
5 tensor(0.6989, device='cuda:0', grad_fn=<NllLossBackward>)
6 tensor(0.6792, device='cuda:0', grad_fn=<NllLossBackward>)
7 tensor(0.6599, device='cuda:0', grad_fn=<NllLossBackward>)
8 tensor(0.6410, device='cuda:0', grad_fn=<NllLossBackward>)
9 tensor(0.6226, device='cuda:0', grad_fn=<NllLossBackward>)
10 tensor(0.6046, device='cuda:0', grad_fn=<NllLossBackward>)
11 tensor(0.5871, device='cuda:0', grad_fn=<NllLossBackward>)
12 tensor(0.5700, device='cuda:0', grad_fn=<NllLossBackward>)
13 tensor(0.5534, device='cuda:0', grad_fn=<NllLossBackward>)
14 tensor(0.5373, device='cuda:0', grad_fn=<NllLossBackward>)
15 tensor(0.5216, device='cuda:0', grad_fn=<NllLossBackward>)
16 tensor(0.5063, 

KeyboardInterrupt: 

In [5]:
AF_ECG = np.load(f'{data_path}/AF_v5/{split}_ECG_resampled2400.npy')
# PVC_ECG = np.load(f'{data_path}/PVC_v5/{split}_ECG_resampled2400.npy')
# NSR_ECG = np.load(f'{data_path}/NSR_v5/{split}_ECG_resampled2400.npy')

In [43]:
issue = AF_ECG[203889]

In [44]:
issue

array([nan, nan, nan, ..., nan, nan, nan])

In [55]:
np.argwhere(np.isnan(AF_ECG))

array([[203889,      0],
       [203889,      1],
       [203889,      2],
       ...,
       [203889,   2397],
       [203889,   2398],
       [203889,   2399]])

In [56]:
np.argwhere(np.isnan(PVC_ECG))

array([], shape=(0, 2), dtype=int64)

In [59]:
np.argwhere(np.isnan(NSR_ECG))

array([], shape=(0, 2), dtype=int64)

In [60]:
np.argwhere(np.isinf(AF_ECG))

array([], shape=(0, 2), dtype=int64)

In [61]:
np.argwhere(np.isinf(PVC_ECG))

array([], shape=(0, 2), dtype=int64)

In [62]:
np.argwhere(np.isinf(NSR_ECG))

array([], shape=(0, 2), dtype=int64)

(2757889, 2400)
(2757888, 2400)


In [2]:
data_path = '/labs/hulab/stark_stuff/ppg_ecg_project/data/'
split = 'train'
AF_ECG = np.load(f'{data_path}/AF_v5/{split}_ECG_resampled2400.npy')
print(AF_ECG.shape)


(2757889, 2400)


In [3]:
AF_PPG = np.load(f'{data_path}/AF_v5/{split}_PPG_resampled2400.npy')

In [4]:
print(AF_ECG.shape)
AF_ECG = np.delete(AF_ECG, 203889, axis=0)
print(AF_ECG.shape)

(2757889, 2400)
(2757888, 2400)


In [5]:
np.save(f'{data_path}/AF_v5/{split}_ECG_resampled2400.npy', AF_ECG)

In [6]:
print(AF_PPG.shape)
AF_PPG = np.delete(AF_PPG, 203889, axis=0)
print(AF_PPG.shape)

(2757889, 2400)
(2757888, 2400)


In [ ]:
np.save(f'{data_path}/AF_v5/{split}_PPG_resampled2400.npy', AF_PPG)